In [1]:
from langchain.document_loaders import RSSFeedLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
urls = ["https://medium.com/feed/tag/quantum-computing"]

In [3]:
loader = RSSFeedLoader(urls=urls)
data = loader.load()
print(len(data))

10


In [4]:
data

[Document(page_content='Investing in DeepTech\n\nDie Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft genutzt, um Geschäftsmodelle zu beschreiben die eher etablierte und wenig innovative Standard-Technologien nutzen. Von HighTech spricht man dann eher bei innovativen Technologien, die sich aber noch im Bereich des klassischen Engineerings bewegen, wogegen im DeepTech-Bereich neuste wissenschaftliche Entwicklungen und Erkenntnisse in Produkte und Geschäftsmodelle umgesetzt werden. Oft sind die HighTech und die DeepTech Themen oft auch mit dem Einsatz von viel und teurer Hardware verbunden und die Startup-Investments sind entsprechend kapitalintensiv.\n\n“Zu einem guten DeepTech Investment gehört neben einer hoch-innovativ und unique Technologie an sich auch, dass diese Technologie zumindest mittel- bis langfristig die Grundlage für einen nachha

In [5]:
print(data[0].page_content)

Investing in DeepTech

Die Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft genutzt, um Geschäftsmodelle zu beschreiben die eher etablierte und wenig innovative Standard-Technologien nutzen. Von HighTech spricht man dann eher bei innovativen Technologien, die sich aber noch im Bereich des klassischen Engineerings bewegen, wogegen im DeepTech-Bereich neuste wissenschaftliche Entwicklungen und Erkenntnisse in Produkte und Geschäftsmodelle umgesetzt werden. Oft sind die HighTech und die DeepTech Themen oft auch mit dem Einsatz von viel und teurer Hardware verbunden und die Startup-Investments sind entsprechend kapitalintensiv.

“Zu einem guten DeepTech Investment gehört neben einer hoch-innovativ und unique Technologie an sich auch, dass diese Technologie zumindest mittel- bis langfristig die Grundlage für einen nachhaltigen wirtschaftlichen Erfo

In [6]:
data[0].metadata

{'title': 'Investing in DeepTech',
 'link': 'https://medium.com/deeptech-climate-fonds/investing-in-deeptech-4d70e7fb8ced?source=rss------quantum_computing-5',
 'authors': ['Deeptech', 'Climate Fonds'],
 'language': 'en',
 'description': 'Die Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft…',
 'publish_date': datetime.datetime(2024, 2, 21, 10, 45, 32, 552000, tzinfo=tzutc()),
 'feed': 'https://medium.com/feed/tag/quantum-computing'}

In [7]:
import sys 
sys.path.append(r'C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding')
from utils import QDrantDBManager, EmbeddingFunction, TextSplitter

In [8]:
embedding = EmbeddingFunction('openAI').embedder

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)

Collection rio-rag-platform already exists!


In [10]:
textSplitter=TextSplitter()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [12]:
splitted_docs=[]
for doc in data: 
    
    splitted_docs.append(text_splitter.split_documents(data))

In [13]:
len(splitted_docs)

10

In [14]:
splitted_docs[0]

[Document(page_content='Investing in DeepTech\n\nDie Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft genutzt, um Geschäftsmodelle zu beschreiben die eher etablierte und wenig innovative Standard-Technologien nutzen. Von HighTech spricht man dann eher bei innovativen Technologien, die sich aber noch im Bereich des klassischen Engineerings bewegen, wogegen im DeepTech-Bereich neuste wissenschaftliche Entwicklungen und Erkenntnisse in Produkte und Geschäftsmodelle umgesetzt werden. Oft sind die HighTech und die DeepTech Themen oft auch mit dem Einsatz von viel und teurer Hardware verbunden und die Startup-Investments sind entsprechend kapitalintensiv.\n\n“Zu einem guten DeepTech Investment gehört neben einer hoch-innovativ und unique Technologie an sich auch, dass diese Technologie zumindest mittel- bis langfristig die Grundlage für einen nachha

In [15]:
splitted_docs[0][0].metadata

{'title': 'Investing in DeepTech',
 'link': 'https://medium.com/deeptech-climate-fonds/investing-in-deeptech-4d70e7fb8ced?source=rss------quantum_computing-5',
 'authors': ['Deeptech', 'Climate Fonds'],
 'language': 'en',
 'description': 'Die Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft…',
 'publish_date': datetime.datetime(2024, 2, 21, 10, 45, 32, 552000, tzinfo=tzutc()),
 'feed': 'https://medium.com/feed/tag/quantum-computing'}

In [16]:
#filter metadata to make it json serializable
import datetime
def filter_datetime_metadata(docs):
    for doc in docs:
        doc.metadata['source'] = 'rss'
        if isinstance(doc.metadata['publish_date'], datetime.datetime):
            # print(doc.metadata['publish_date'])
            doc.metadata['publish_date'] = doc.metadata['publish_date'].strftime("%Y-%m-%d")

In [17]:
filter_datetime_metadata(splitted_docs[0])

In [18]:
splitted_docs[0][0].metadata

{'title': 'Investing in DeepTech',
 'link': 'https://medium.com/deeptech-climate-fonds/investing-in-deeptech-4d70e7fb8ced?source=rss------quantum_computing-5',
 'authors': ['Deeptech', 'Climate Fonds'],
 'language': 'en',
 'description': 'Die Begriffe DeepTech, HighTech und LowTech sind leider nicht wirklich trennscharf definiert und werden gerade im Investment Bereich oft subjektiv und sehr unterschiedlich benutzt. LowTech wird oft…',
 'publish_date': '2024-02-21',
 'feed': 'https://medium.com/feed/tag/quantum-computing',
 'source': 'rss'}

In [20]:
qdrantClient.index_documents(splitted_docs[0])